In [145]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.{VectorAssembler, IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.linalg.{Vector, VectorUDT}

val spark = SparkSession
  .builder()
  .appName("Scala context")
  .getOrCreate()

spark = org.apache.spark.sql.SparkSession@79f1bbe0


org.apache.spark.sql.SparkSession@79f1bbe0

# Train

In [2]:
val train = spark.read
    .options(Map("inferSchema"->"true","delimiter"->",","header"->"true"))
    .csv("/labs/slaba05/lab05_train.csv")
    .filter("TARGET is not NULL")

train = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [99]:
val selected_cols = train.dtypes.slice(2, 116).filter(_._2 != "StringType").map(m => m._1)//.length

selected_cols = Array(CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, CNT_TRAN_MED_TENDENCY1M, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, SUM_TRAN_CLO_TENDENCY1M, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CNT_TRAN_MED_TENDENCY3...


Array(CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, CNT_TRAN_MED_TENDENCY1M, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, SUM_TRAN_CLO_TENDENCY1M, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CNT_TRAN_MED_TENDENCY3...

In [100]:
val assembler = new VectorAssembler()
    .setInputCols(selected_cols)
    .setOutputCol("features")
    .setHandleInvalid("keep")

assembler = vecAssembler_b73289a6a2d5


vecAssembler_b73289a6a2d5

In [101]:
val feat_df = assembler.transform(train).select("features", "TARGET")

feat_df = [features: vector, TARGET: int]


[features: vector, TARGET: int]

In [102]:
val Array(train_df, valid_df) = feat_df.randomSplit(Array(0.7, 0.3))

train_df = [features: vector, TARGET: int]
valid_df = [features: vector, TARGET: int]


[features: vector, TARGET: int]

In [103]:
val gbt = new GBTClassifier()
  .setLabelCol("TARGET")
  .setFeaturesCol("features")
  .setMaxIter(10)
  .setFeatureSubsetStrategy("auto")

gbt = gbtc_9172eeb70c70


gbtc_9172eeb70c70

In [104]:
val model = gbt.fit(train_df)

model = GBTClassificationModel (uid=gbtc_9172eeb70c70) with 10 trees


GBTClassificationModel (uid=gbtc_9172eeb70c70) with 10 trees

In [105]:
val train_predictions = model.transform(train_df)
val valid_predictions = model.transform(valid_df)

train_predictions = [features: vector, TARGET: int ... 3 more fields]
valid_predictions = [features: vector, TARGET: int ... 3 more fields]


[features: vector, TARGET: int ... 3 more fields]

In [107]:
val evaluator = new BinaryClassificationEvaluator()
  .setLabelCol("TARGET")
  .setRawPredictionCol("rawPrediction")
val train_roc_auc = evaluator.evaluate(train_predictions)
val valid_roc_auc = evaluator.evaluate(valid_predictions)

evaluator = binEval_8fa85ae62ec6
train_roc_auc = 0.8048769114304277
valid_roc_auc = 0.8000631425161451


0.8000631425161451

# Inference

In [109]:
val test = spark.read
    .options(Map("inferSchema"->"true","delimiter"->",","header"->"true"))
    .csv("/labs/slaba05/lab05_test.csv")

test = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [115]:
val test_df = assembler.transform(test).select("ID","features")

test_df = [ID: int, features: vector]


[ID: int, features: vector]

In [156]:
val tailValue = udf((v: Vector) => v.toArray(1))

val train_predictions = model.transform(test_df)
    .select(col("ID").alias("id"), tailValue(col("probability")).alias("target"))
    .repartition(1)

tailValue = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
train_predictions = [id: int, target: double]


[id: int, target: double]

In [157]:
train_predictions.show(5)

+------+-------------------+
|    id|             target|
+------+-------------------+
|332855|0.06887325146961143|
|587122|0.07605410758299935|
|523949| 0.2005944265486388|
|363029|0.09400420479099902|
|259664|0.19410564661236696|
+------+-------------------+
only showing top 5 rows



In [162]:
train_predictions.write
    .mode("overwrite")
    .option("header",true)
    .option("delimiter","\t")
    .csv("lab05.csv")